In [11]:
import pandas as pd
import numpy as np
import tensorflow as tf
import sys
from PIL import Image
from matplotlib import pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Convolution2D, Dropout, Flatten, Dense, Input# Sequential model for building a linear stack of layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16

In [3]:
sys.path.append('/Users/jordankanius/LHL_projects/Face2Face_Real_vs_Fake/functions/')

In [4]:
from create_dataframe import create_df
train_meta = create_df('train')
valid_meta = create_df('valid')
test_meta = create_df('test')

In [5]:
# Specify the base path where your image data is stored
base_path = '../real_vs_fake/real-vs-fake/'

# Create an ImageDataGenerator for image preprocessing and augmentation
image_gen = ImageDataGenerator(rescale=1./255.)
# The 'rescale' parameter normalizes pixel values to the range [0, 1] by dividing each pixel value by 255

In [6]:
VGG16_train = image_gen.flow_from_directory(
    base_path + 'train/',
    target_size=(256, 256),
    batch_size=100,
    class_mode='binary'
)
VGG16_valid = image_gen.flow_from_directory(
    base_path + 'valid/',
    target_size=(256, 256),
    batch_size=100,
    class_mode='binary'
)
VGG16_test = image_gen.flow_from_directory(
    base_path + 'test/',
    target_size=(256, 256),
    batch_size=100,
    class_mode='binary'
)

Found 100000 images belonging to 2 classes.
Found 20000 images belonging to 2 classes.
Found 20000 images belonging to 2 classes.


In [8]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(256, 256, 3))

58889256/58889256 [==============================] - 2s 0us/step


In [12]:
input_shape = (256, 256, 3)
model = Sequential()
model.add(Input(shape=input_shape))
model.add(base_model)
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 8, 8, 512)         14714688  
                                                                 
 dropout (Dropout)           (None, 8, 8, 512)         0         
                                                                 
 flatten (Flatten)           (None, 32768)             0         
                                                                 
 dense (Dense)               (None, 64)                2097216   
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 16,811,969
Trainable params: 16,811,969
Non-trainable params: 0
_________________________________________________________________


In [14]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [15]:
model.fit(VGG16_train, epochs= 10, validation_data=VGG16_valid)

Epoch 1/10


2023-12-08 20:34:07.183211: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


   1/1000 [..............................] - ETA: 14:37:50 - loss: 0.7090 - accuracy: 0.5800


KeyboardInterrupt

